In [ ]:
using Pkg; Pkg.activate("..")
using Plots

using Random, LinearAlgebra

using Revise
push!(LOAD_PATH, String(@__DIR__)*"/src")
using Tracks
using Position: TPoint, is_inside, Box

In [ ]:
e1 = TPoint(0,0,0)
e2 = TPoint(3000,3000,4000)
box = Box(e1, e2)

In [ ]:
function f(x, center, width, intercept = -125)
    x = x-center
    a = (3000-intercept)/(width/2)^2
    a*x^2+intercept
end

function my_valley(x, y)
    ledge = 1000+200*sin(y/200)
    redge = 2500+300*cos(y/750)
    
    width = redge-ledge
    center = (ledge + redge)/2
    
    if x < ledge || x > redge
        height = 3000
    else
        height = maximum((0, f(x, center, width)))
    end
    
    height += maximum((0, ((y)^2/1000+2500)*(abs(x-center)/25)^-1))
#     height += maximum((0, (-(y-3000)^2/1000+2500)*(abs(x-center)/25)^-1))
    height = minimum((height, 3000))
end

my_valley(1200, 3423)

In [ ]:
function is_in_mountain(x, y, z)
    is_inside(x, y, z, my_valley)
end

function is_in_mountain(p::TPoint)
    is_in_mountain(p.x, p.y, p.z)
end    

function is_in_mountain(t::Track)
    p = t.ipoint
    is_in_mountain(p)
end

In [ ]:
n_points = 100
twista = MersenneTwister(925)

#=
Generate initial points for the tracks to start at
These should be insidee the box
and be uniformly distributed in y and z
And be towards the low-x end of the box
=#
xx = rand(twista, n_points).*500 .-250 .+500
yy = rand(twista, n_points).*3000
zz = rand(twista, n_points).*3000
ipointt = [TPoint(x...) for x in zip(xx,yy,zz)]
#=
Sanity chack to make sure that all points are in the box
This should be true by construction
If this fails, it could mean the point generation is bad
or, more likely, the is_inside function is meessed up
=#
for pt in ipointt
    if !is_inside(pt, box)
        println("Oh no ! The initial pointis outside the box")
    end
end
#=
Generate initial directions for the tracks
These should roughly be pointing horizontal and to the right
i.e. θ∼π/2 and ϕ∼0 
=#
θθ = randn(twista, n_points) ./ 4 .+ π/2
ϕϕ = randn(twista, n_points) ./ 5 .- 0.25

# Make the tracks
trackk  = [Track(x...) for x in zip(ipointt, ϕϕ, θθ)]
# Normalize them to the box
ntrackk = [normalize_track(track, box) for track in trackk if is_in_mountain(track)];

In [ ]:
# An homage
mutable struct Trayectory
    track::Track
    dλ::Float64
    x::Vector{Float64}
    y::Vector{Float64}
    z::Vector{Float64}
    current_λ
    function Trayectory(track::Track, dλ::Float64)
        new(track, dλ, [], [], [], 0.0)
    end
end

function step!(t::Trayectory)
    t.current_λ += t.dλ
    p = find_position(t.track, t.current_λ)
    append!(t.x, p.x)
    append!(t.y, p.y)
    append!(t.z, p.z)
end

function split_for_plot(x, y, z)
    region_change = diff([is_in_mountain(x...) for x in zip(x, y, z)]).!=0
    split_i       = [i for i in 1:length(region_change) if region_change[i]==1]
    split_track   = []
    old_idx       = 1
    current_color = ifelse(is_in_mountain(x[1], y[1], z[1]), :red, :black)
    for new_idx in split_i
        push!(split_track, (x[old_idx:new_idx], y[old_idx:new_idx], z[old_idx:new_idx], current_color))
        old_idx = new_idx
        current_color = ifelse(current_color==:black, :red, :black)
    end
    push!(split_track, tuple(x[old_idx:end], y[old_idx:end], z[old_idx:end], current_color))
    split_track
end

function split_for_plot(t::Trayectory)
    split_for_plot(t.x, t.y, t.z)
end

In [ ]:
tr = ntrackk[3]

res         = 200
gifify      = true
show_track  = false
spatial_res = 20 # meter
dλ          = spatial_res/norm(tr.direction)

xx = LinRange(0, 3000, res)
yy = LinRange(0, 3000, res)


kwargs = Dict(:xlim=>(minimum((e1.x, e2.x)), maximum((e1.x, e2.x))),
          :ylim=>(minimum((e1.y, e2.y)), maximum((e1.y, e2.y))),
          :zlim=>(minimum((e1.z, e2.z)), maximum((e1.z, e2.z))),
          :st=>:surface,
          :alpha=>0.9,
          :c=>cgrad(palette([:skyblue3, :skyblue2, :navajowhite3, :navajowhite3, :goldenrod4, :goldenrod4, :olivedrab, :olivedrab, :green, :green, :green, :green])),
          :colorbar=>false,
          :legend=>false,
         )

l   = @layout [a{0.7h} ; c{0.5w} d]
plt = plot3d(xx, yy, my_valley, layout=l, axis=([], false); kwargs...)
# This should be done with an imshow-like thing.... I think
plot!(plt[2], xx, yy, my_valley; camera=(0,90), kwargs...)
plot!(plt[3], xx, yy, my_valley; camera=(0,0), kwargs...)

if show_track
    λλ  = LinRange(0, 1, 50)
    pts = [tr.ipoint+λ*tr.direction for λ in λλ]
    fuck = split_for_plot([getfield.(pts, s) for s in [:x, :y, :z]]...)
    for f in fuck
        for idx in 1:3
            plot!(plt[idx], f[1:3]..., color=f[4], legend=false)
        end
    end
end

display(plt)
Plots.savefig(plt, "/Users/jlazar/research/TAMBO-MC/figures/fake_valley.pdf")

if gifify && ~show_track # These options don't play well together. Sorry
trayectory = Trayectory(tr, dλ)    
    @gif while trayectory.current_λ <=1
        step!(trayectory)
        fuck = split_for_plot(trayectory)
        for f in fuck
            for idx in 1:3
                plot!(plt[idx], f[1:3]..., color=f[4], legend=false)
            end
        end
        plot!(plt[1], camera=(minimum((90, trayectory.current_λ*100)), minimum((90, trayectory.current_λ*100))))
    end every 1
end